<a href="https://colab.research.google.com/github/financieras/campus/blob/main/pilas/pilas28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo Deep Moder Times preservando los tres últimos números en A
* Solo usando parejas para pasar de A a B los primeros números
* Al llegar a los tres últimos números en A, se ordenan creciente circular
* Se retornan todos los números de B a A, pero de forma dirigida, encajándolos entre los de A
* Ahora B está vacía y solo queda poner el mínimo de A el primero

In [1]:
# FUNCIONES
def sa(a,b):
    if len(a) > 1: a[0],a[1] = a[1],a[0]
    return a,b
def sb(a,b):
    if len(b) > 1: b[0],b[1] = b[1],b[0]
    return a,b
def ss(a,b):
    sa(a,b)
    sb(a,b)
    return a,b
def pa(a,b):
    if len(b) > 0:
        a.insert(0, b[0])
        b.pop(0)
    return a,b
def pb(a,b):
    if len(a) > 0:
        b.insert(0, a[0])
        a.pop(0)
    return a,b
def ra(a,b):
    if len(a) > 1: a.append(a.pop(0))
    return a,b
def rb(a,b):
    if len(b) > 1: b.append(b.pop(0))
    return a,b
def rr(a,b):
    ra(a,b)
    rb(a,b)
    return a,b
def rra(a,b):
    if len(a) > 1: a.insert(0, a.pop())
    return a,b
def rrb(a,b):
    if len(b) > 1: b.insert(0, b.pop())
    return a,b
def rrr(a,b):
    rra(a,b)
    rrb(a,b)
    return a,b

In [2]:
# PASOS NECESARIOS PARA COLOCAR CADA ELEMENTO DE A EN SU SITIO EN B      total = pasosA + pasosB (esta suma es solo una idea, se ha de sumar de una forma peculiar)

def necesariosA(a, b):   # array pasosA calcula los pasos necesarios para colocar cada elemento de A como el primero de su pila
    for v in a:
        if a.index(v) < len(a)/2:
            pasosA.append(a.index(v))
        else:
            pasosA.append(-(len(a)- a.index(v)))

def necesariosB(a,b):   # array pasosB calcula los pasos de B necesarios para colocar cada elemento de A dentro de su sitio en B
    for i in range(len(a)):    # objetivo_primero es el número que se ha de poner en la primera posición de la pila B
        if a[i] < min(b):      # si el elemento de A considerado es menor que el mayor de B entonces
            objetivo_primero = max(b)   # el objetivo_primero será el mayor de B
        else:   # objetivo_primero en este caso será el maximo de los inferiores en B al valor iésimo de A
            objetivo_primero = min(b)   #se inicializa en el valor mínimo de la pila B
            for j in range(len(b)):
                if b[j] < a[i] and b[j] > objetivo_primero:
                        objetivo_primero = b[j]
        # el objetivo_primero se ha de situar el primero de la pila B            
        if b.index(objetivo_primero) < len(b)/2:
            pasosB.append(b.index(objetivo_primero))
        else:
            pasosB.append(-(len(b)- b.index(objetivo_primero)))

def totaliza(a,b):   # totalizar pasos
    global total
    for i in range(len(pasosA)):
        if pasosA[i] * pasosB[i] < 0:   # si son de distinto signo, uno positivo y otro negativo
            total.append(abs(pasosA[i]) + abs(pasosB[i]))   # no hay sinergia
        else:   # si son de igual signo o alguno cero
            total.append(max(abs(pasosA[i]), abs(pasosB[i])))  # si son de igual signo hay sinergia

def calculaIndexPasosMinimos():
    global pasosA, pasosB, total
    pasosA = []   # [0,1,2, ...,41,-41,... , -2,-1]  vector donde cada index está asociado con el valor del mismo index en A
    pasosB = []   # [-3,2,-5, ..., -5,0]   estos dos arrays se han de recalcular cada vez que realmente se mueva algún elemento de A a B
    total = []
    necesariosA(a, b)
    necesariosB(a, b)
    totaliza(a,b)
    return total.index(min(total))   # retorna el índice del elemento de la pila A que menos pasos totales necesita

In [3]:
def giraA(a, b, indice, pasos):
    global contador
    for i in range(abs(pasos)):
        if pasos > 0:
            ra(a,b); contador += 1
        elif pasos < 0:
            rra(a,b); contador += 1

def giraB(a, b, indice, pasos):
    global contador
    for i in range(abs(pasos)):
        if pasos > 0:
            rb(a,b); contador += 1
        elif pasos < 0:
            rrb(a,b); contador += 1

# girando pilas A y B
def giraPilas(a,b,indice):   # indice es el index del valor en la pila A que deseamos poner el primero
    global contador
    if pasosA[indice] * pasosB[indice] > 0:   # Existe sinergia, nos podemos ahorrar pasos
        pasos_comunes = min(abs(pasosA[indice]), abs(pasosB[indice]))
        for i in range(pasos_comunes):
            if pasosA[indice] > 0:   # si el signo de ambos es positivo, ya que ambos tienen el mismo signo
                rr(a,b); contador += 1
            elif pasosA[indice] < 0: # si el signo de ambos es negativo
                rrr(a,b); contador += 1
        exceso_pasosA = abs(pasosA[indice]) - pasos_comunes
        exceso_pasosB = abs(pasosB[indice]) - pasos_comunes
        giraA(a, b, indice, ((pasosA[indice] > 0) - (pasosA[indice] < 0)) * exceso_pasosA)    # (a > 0) - (a < 0) da el signo de a
        giraB(a, b, indice, ((pasosB[indice] > 0) - (pasosB[indice] < 0)) * exceso_pasosB)   # Python no tiene función sign
    else:   # No existe sinergia
        giraA(a ,b, indice, pasosA[indice])   # gira A
        giraB(a, b, indice, pasosB[indice])   # gira B

In [4]:
def crecientesA(a,b):   # función que comprueba que en A están crecientes estrictos y sino lo consigue
    global contador
    if len(a) != 3:
        print("ERROR: en A no hay tres números") 
    elif a[0] < a[1] > a[2] and a[0] < a[2]:        # caso: 1 3 2
        ra(a,b); contador += 1
        sa(a,b); contador += 1
        rra(a,b); contador += 1
    elif a[0] > a[1] < a[2] and a[0] < a[2]:        # caso: 2 1 3
        sa(a,b); contador += 1
    elif a[0] < a[1] > a[2] and a[0] > a[2]:        # caso: 2 3 1
        rra(a,b); contador += 1
    elif a[0] > a[1] < a[2] and a[0] > a[2]:        # caso: 3 1 2
        ra(a,b); contador += 1
    elif a[0] > a[1] > a[2]:                        # caso: 3 2 1
        sa(a,b); contador += 1
        rra(a,b); contador += 1
    return                                          # caso: 1 2 3 en este caso no se hace nada pq ya están ordenados

In [5]:
def situarMax_en_B():   # situa el valor máximo de B en la primera posición de B
    indice = b.index(max(b))
    if indice < len(a)/2:
        pasos = indice
    else:
        pasos = -(len(b)- indice)
    giraB(a, b, indice, pasos)

def cremallera(a,b):   # función que va pasando los números de B a A en forma de cremallera, intercalándolos con los que ya hay en A
    global contador
    aux = [a[-3], a[-2], a[-1]]   # los tres últimos valores de A se copian a una array auxiliar
    for i in range(3):
        while b[0] > max(aux):
            pa(a,b); contador += 1
        rra(a,b); contador += 1
        aux.pop()
    while len(b) > 0:
        pa(a,b); contador += 1

In [ ]:
if __name__ == "__main__":
    from random import seed, shuffle
    seed()
    n = 500   # número de elementos de la pila
    contadores = []
    for muestra in range(1):
        a = list(range(1, n+1)); shuffle(a)   # generación aleatoria de la pila A
        a = [474, 188, 52, 468, 132, 334, 412, 434, 141, 212, 420, 143, 80, 60, 218, 329, 376, 109, 485, 213, 230, 65, 461, 257, 106, 282, 440, 190, 322, 358, 231, 209, 49, 281, 202, 245, 333, 239, 354, 352, 445, 388, 146, 298, 21, 54, 233, 304, 289, 470, 56, 272, 90, 22, 127, 119, 208, 136, 174, 236, 273, 130, 347, 293, 424, 157, 43, 310, 118, 266, 197, 95, 493, 31, 326, 53, 235, 185, 463, 379, 242, 287, 256, 264, 394, 71, 346, 292, 79, 345, 277, 477, 211, 162, 398, 362, 126, 274, 45, 34, 370, 300, 339, 97, 234, 240, 448, 37, 87, 9, 330, 489, 343, 222, 204, 124, 207, 238, 491, 267, 414, 288, 480, 166, 351, 50, 184, 152, 367, 258, 8, 260, 153, 286, 454, 133, 285, 104, 500, 254, 361, 129, 389, 380, 299, 117, 372, 303, 223, 279, 432, 422, 32, 386, 262, 122, 228, 395, 460, 28, 59, 108, 308, 495, 203, 349, 302, 441, 382, 224, 155, 205, 237, 101, 121, 449, 210, 48, 317, 413, 336, 366, 451, 476, 67, 13, 88, 364, 219, 436, 147, 139, 195, 148, 360, 423, 30, 408, 416, 296, 85, 473, 453, 225, 456, 271, 496, 39, 250, 275, 487, 455, 447, 27, 261, 335, 110, 353, 374, 430, 409, 269, 312, 247, 115, 253, 206, 483, 149, 259, 472, 280, 168, 309, 406, 221, 145, 392, 357, 75, 444, 419, 16, 69, 481, 435, 373, 484, 446, 467, 113, 399, 15, 41, 29, 268, 490, 215, 189, 63, 47, 102, 62, 438, 307, 128, 443, 402, 83, 325, 290, 36, 193, 381, 25, 70, 58, 393, 278, 73, 452, 131, 156, 111, 26, 163, 431, 305, 276, 154, 138, 478, 142, 38, 150, 457, 4, 342, 363, 92, 401, 311, 385, 319, 378, 220, 265, 159, 340, 2, 137, 471, 355, 180, 251, 494, 103, 6, 301, 86, 397, 10, 306, 14, 192, 426, 246, 116, 125, 323, 151, 321, 295, 263, 123, 294, 368, 249, 20, 465, 158, 112, 338, 391, 405, 18, 429, 475, 383, 421, 199, 344, 479, 61, 194, 359, 17, 499, 164, 324, 120, 375, 229, 365, 320, 179, 403, 64, 196, 270, 191, 466, 19, 183, 200, 488, 283, 410, 400, 76, 23, 172, 415, 160, 315, 140, 350, 337, 165, 462, 437, 78, 482, 227, 169, 186, 105, 313, 40, 328, 433, 144, 217, 459, 134, 89, 93, 458, 201, 464, 369, 135, 46, 82, 291, 96, 5, 216, 114, 3, 243, 469, 486, 241, 98, 442, 407, 77, 492, 427, 175, 498, 100, 284, 84, 11, 35, 371, 244, 161, 167, 396, 214, 181, 390, 418, 232, 182, 384, 66, 94, 341, 331, 377, 91, 450, 318, 44, 176, 497, 55, 1, 411, 226, 404, 255, 198, 297, 81, 173, 356, 170, 178, 187, 74, 428, 12, 51, 387, 327, 332, 248, 425, 417, 348, 252, 68, 314, 33, 439, 171, 24, 316, 99, 72, 57, 7, 177, 107, 42]
        a_original = a[:]
        b = []
        contador = 0    # contador de pasos
        pb(a,b); contador += 1   # bajamos el primer número de A a B
        pb(a,b); contador += 1   # bajamos el segundo número de A a B
        while len(a)>3:
            index_minimosPasos = calculaIndexPasosMinimos()
            giraPilas(a, b, index_minimosPasos)
            pb(a,b); contador += 1 # lo pasa de A a B haciendo pb
        crecientesA(a,b)   # función que comprueba que en A están crecientes estrictos y sino lo consigue
        situarMax_en_B()   # situa el valor máximo de B en la primera posición de B
        cremallera(a,b)    # función que va pasando los números de B a A en forma de cremallera, intercalándolos con los que ya hay en A
        contadores.append(contador)
    print(contadores)
    print("maximo contadores: ", max(contadores))

Idea 1
* Cuando la pila A tiene n/2 datos buscar la zona más amplia de números crecientes en posiciones seguidas, por ejemplo 2, 18, 23, 32, 39, 44
* Preservar esa zona para el final


Idea 2
* al girarA o girarB aprovechar las ordenes ra rb o rra rrb para ver si interesa meter comodines con el objeto de acercar valores a otros más cercanos o idealmente contiguos 